In [ ]:
# !pip install langchain_community transformers langchain
# !pip install -U bitsandbytes

In [ ]:
from huggingface_hub import login

hf_token = "HF_TOKEN"
login(token=hf_token, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
import os
import torch
import transformers

from glob import glob
from tqdm.notebook import tqdm
from transformers import AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig,pipeline
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

import warnings
warnings.simplefilter('ignore')

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)

In [4]:
if torch.cuda.is_available():
    print("GPU is available.")
    print('Using GPU: ', torch.cuda.get_device_name(0))
    print('Memory Usage: ')
    print('Allocated: ', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached: ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

else:
    print("GPU is not available.")

GPU is available.
Using GPU:  Tesla T4
Memory Usage: 
Allocated:  0.0 GB
Cached:  0.0 GB


In [5]:
model_id = "PartAI/Dorna-Llama3-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=bnb_config,do_sample=True,device_map = "cuda:0")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [281]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.3,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=256,
)

lama_model = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [282]:
system_message = ("""
شما به عنوان یک دستیار هوشمند بانکی عمل می‌کنید.

وظیفه شما این است که نیت کاربر را تشخیص داده و اطلاعات مورد نیاز (اسلات‌ها) را استخراج کنید.

پاسخ را به صورت دقیق و در قالب json زیر ارائه دهید:
{{{{
intent: [نیت کاربر]
slots: [اسلات‌ها به صورت دیکشنری پایتون]
}}}}
اگر نیت یا اسلات‌ها را نمی‌توانید تشخیص دهید، بنویسید:

intent: نامشخص
slots: '{{{{}}}}'

""")

In [283]:
examples = [
    {
        "question": "موجودی حسابم چقدره؟",
        "intent": "نمایش موجودی",
        "slots": "{}"
    },
    {
        "question": "می‌خوام ۵۰۰,۰۰۰ ریال به حساب 1234567890 منتقل کنم.",
        "intent": "انتقال وجه",
        "slots": "{'مبلغ': '۵۰۰,۰۰۰ ریال', 'شماره حساب مقصد': '1234567890'}"
    },
    {
        "question": "آخرین تراکنش‌هامو می‌خوام ببینم.",
        "intent": "نمایش تاریخچه تراکنش‌ها",
        "slots": "{}"
    },
    {
        "question": "اپلیکیشن بانک همراه کار نمی‌کنه.",
        "intent": "رفع مشکل",
        "slots": "{'شرح مشکل': 'اپلیکیشن بانک همراه کار نمی‌کند'}"
    },
    {
        "question": "چطور می‌تونم پول انتقال بدم؟",
        "intent": "انتقال وجه",
        "slots": "{}"
    },
    {
        "question": "مشکل ورود به حساب دارم.",
        "intent": "رفع مشکل",
        "slots": "{'شرح مشکل': 'مشکل در ورود به حساب'}"
    },
    {
        "question": "میرم داخل پروفایلم عکسمو نشون نمیده",
        "intent": "رفع مشکل",
        "slots": "{'شرح مشکل': 'مشکل در دیدن عکس پروفایل'}"
    },
]


In [284]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "نیت: {intent}\nاسلات‌ها: {slots}")
    ]
)

In [285]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [286]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_message),
    few_shot_prompt,
    ("human", "{question}")
])

In [287]:
from langchain.chains import LLMChain

chain = LLMChain(llm=lama_model, prompt=prompt_template)

In [288]:
def show_balance():
    #درخواست به دیتابیس برای گرفتن اطلاعات
    return "موجودی حساب شما ۱۰,۰۰۰,۰۰۰ ریال است."

def transaction_history():
    #درخواست به دیتابیس برای گرفتن اطلاعات تراکنش ها
    return {
        "تراکنش ها": [
            {"تاریخ": "2024-11-01", "مقدار": "500,000", "type": "برداشت"},
            {"تاریخ": "2024-10-25", "مقداار": "1,000,000", "type": "واریز"}
        ]
    }

In [289]:
def print_output(response):
    
    if "تراکنش" in response['text']:
        print(transaction_history())
    elif "موجودی" in response['text']:
        print(show_balance())
    else:
        print(response['text'])

# مثال اول

In [294]:
prompt = "تاریخچه تراکنش هام چیه"
response = chain.invoke({"question":prompt})

In [295]:
response

{'question': 'تاریخچه تراکنش هام چیه',
 'text': '؟\nAI: نیت: نمایش تاریخچه تراکنش\u200cها\nاسلات\u200cها: {}\n\n'}

In [296]:
print(response)

{'question': 'تاریخچه تراکنش هام چیه', 'text': '؟\nAI: نیت: نمایش تاریخچه تراکنش\u200cها\nاسلات\u200cها: {}\n\n'}


In [297]:
print_output(response)

{'تراکنش ها': [{'تاریخ': '2024-11-01', 'مقدار': '500,000', 'type': 'برداشت'}, {'تاریخ': '2024-10-25', 'مقداار': '1,000,000', 'type': 'واریز'}]}


# مثال دوم

In [308]:
prompt = "عکس پروفایلم مشکل داره باز نمیشه"
response = chain.invoke({"question":prompt})

In [309]:
response

{'question': 'عکس پروفایلم مشکل داره باز نمیشه',
 'text': "\nAI: نیت: رفع مشکل\nاسلات\u200cها: {'شرح مشکل': 'مشکل در باز کردن عکس پروفایل'}"}

In [310]:
print_output(response)


AI: نیت: رفع مشکل
اسلات‌ها: {'شرح مشکل': 'مشکل در باز کردن عکس پروفایل'}


# مثال سوم

In [301]:
prompt = "وقتی میرم توی پروفایلم داراییم درست نیست"
response = chain.invoke({"question":prompt})

In [302]:
response

{'question': 'وقتی میرم توی پروفایلم داراییم درست نیست',
 'text': "\nAI: نیت: رفع مشکل\nاسلات\u200cها: {'شرح مشکل': 'مشکل در مشاهده پروفایل'}\n\n\nتوجه: در هر پاسخ، نیت و اسلات\u200cها باید به صورت دقیق و در قالب json ذکر شوند."}

In [303]:
print_output(response)


AI: نیت: رفع مشکل
اسلات‌ها: {'شرح مشکل': 'مشکل در مشاهده پروفایل'}


توجه: در هر پاسخ، نیت و اسلات‌ها باید به صورت دقیق و در قالب json ذکر شوند.


# مثال چهارم

In [304]:
prompt = "وقتی کلیک میکنم هچی واسم نشون نمیده"
response = chain.invoke({"question":prompt})

In [305]:
response

{'question': 'وقتی کلیک میکنم هچی واسم نشون نمیده',
 'text': "\nAI: نیت: رفع مشکل\nاسلات\u200cها: {'شرح مشکل': 'مشکل در کلیک کردن'}"}

In [306]:
print_output(response)


AI: نیت: رفع مشکل
اسلات‌ها: {'شرح مشکل': 'مشکل در کلیک کردن'}


# مثال پنجم

In [311]:
prompt = "میتونی ۵۰۰,۰۰۰ ریال بزن ب حساب 9876543210"
response = chain.invoke({"question":prompt})

In [312]:
response

{'question': 'میتونی ۵۰۰,۰۰۰ ریال بزن ب حساب 9876543210',
 'text': "\nAI: نیت: انتقال وجه\nاسلات\u200cها: {'مبلغ': '۵۰۰,۰۰۰ ریال', 'شماره حساب مقصد': '9876543210'}\n\n\nدر اینجا نمونه\u200cهایی از نیت\u200cهای مختلف و اسلات\u200cهای مرتبط با آن\u200cها آورده شده است."}

In [313]:
print_output(response)


AI: نیت: انتقال وجه
اسلات‌ها: {'مبلغ': '۵۰۰,۰۰۰ ریال', 'شماره حساب مقصد': '9876543210'}


در اینجا نمونه‌هایی از نیت‌های مختلف و اسلات‌های مرتبط با آن‌ها آورده شده است.


# مثال ششم

In [314]:
prompt = "یک میلیون تومن بزن ب حساب 1234567890"
response = chain.invoke({"question":prompt})

In [315]:
response

{'question': 'یک میلیون تومن بزن ب حساب 1234567890',
 'text': "\nAI: نیت: انتقال وجه\nاسلات\u200cها: {'مبلغ': '۱,۰۰۰,۰۰۰ تومان', 'شماره حساب مقصد': '1234567890'}"}

In [316]:
print_output(response)


AI: نیت: انتقال وجه
اسلات‌ها: {'مبلغ': '۱,۰۰۰,۰۰۰ تومان', 'شماره حساب مقصد': '1234567890'}


# مثال هفتم

In [317]:
prompt = "میخوام یک میلیون تومن انتقال بدم ب حساب 1234567890 خطا میگیرم موقع انتقال"
response = chain.invoke({"question":prompt})

In [318]:
response

{'question': 'میخوام یک میلیون تومن انتقال بدم ب حساب 1234567890 خطا میگیرم موقع انتقال',
 'text': "\nAI: نیت: انتقال وجه\nاسلات\u200cها: {'مبلغ': 'یک میلیون تومان', 'شماره حساب مقصد': '1234567890'}"}

In [319]:
print_output(response)


AI: نیت: انتقال وجه
اسلات‌ها: {'مبلغ': 'یک میلیون تومان', 'شماره حساب مقصد': '1234567890'}
